In [3]:
import pandas as pd
import ast
import gzip


### Extraccion

In [4]:
filas = []

# Abrir el archivo JSON comprimido en gzip
with gzip.open("Datasets\\users_items.json.gz", 'rt', encoding='utf-8') as archivo:
    for linea in archivo:
        # ast.literal_eval() convierte cada línea, que es una cadena de texto, en un objeto de Python
        filas.append(ast.literal_eval(linea))

# Crear el DataFrame
df = pd.DataFrame(filas)

In [4]:
df.head(3)

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."


In [11]:
df.shape

(88310, 5)

### Transformacion de datos

Podemos observar que en el dataframe se encuentra una lista de diccionarios. A continuacion desglosamos la lista convirtiendola en un dataframe y agregamos las demas columnas.

In [5]:
# Transforma la lista de diccionarios 'items' en un dataframe
df_normalizado = pd.json_normalize(filas, record_path='items', meta=['user_id', 'items_count', 'steam_id', 'user_url'])
# filas: lista donde se almacena el json
# record_path: aca va la key de los datos a normalizar (Clave de la lista de diccionarios)
# meta: las columnas que queremos incluir en el df

df_normalizado.head(4)

,item_id,item_name,playtime_forever,playtime_2weeks,user_id,items_count,steam_id,user_url
0,10,Counter-Strike,6,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
1,20,Team Fortress Classic,0,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
2,30,Day of Defeat,7,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
3,40,Deathmatch Classic,0,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...


In [16]:
df_normalizado.info()
print("Filas:{} y columnas:{} ".format(*df_normalizado.shape))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5153209 entries, 0 to 5153208
Data columns (total 8 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   item_id           object
 1   item_name         object
 2   playtime_forever  int64 
 3   playtime_2weeks   int64 
 4   user_id           object
 5   items_count       object
 6   steam_id          object
 7   user_url          object
dtypes: int64(2), object(6)
memory usage: 314.5+ MB
Filas:5153209 y columnas:8 


Verificamos si el dataframe contiene valores nulos

In [6]:
#Verificamos si hay valores nulos
df_normalizado.isnull().sum()

item_id             0
item_name           0
playtime_forever    0
playtime_2weeks     0
user_id             0
items_count         0
steam_id            0
user_url            0
dtype: int64

Verificamos si el dataframe contiene valores duplicados

In [7]:
df_normalizado.duplicated().sum()

59104

In [8]:
# Se eliminan los datos duplicados
df_normalizado.drop_duplicates(keep='first',inplace=True)

Eliminamos las siguientes columnas:
*  steam_id  : Contiene la misma data que user_id
*  playtime_2weeks  : tiempo jugado por 2 semanas (Irrelevante) 
*  user_url  : Tiene la url del usuario (Irrelevante)

In [9]:
df_normalizado.drop(['steam_id','playtime_2weeks','user_url'],axis=1, inplace=True)

In [10]:
df_normalizado.head()

,item_id,item_name,playtime_forever,user_id,items_count
0,10,Counter-Strike,6,76561197970982479,277
1,20,Team Fortress Classic,0,76561197970982479,277
2,30,Day of Defeat,7,76561197970982479,277
3,40,Deathmatch Classic,0,76561197970982479,277
4,50,Half-Life: Opposing Force,0,76561197970982479,277


In [11]:
df_normalizado.shape

(5094105, 5)

In [12]:
# Cantidad de filas donde el tiempo jugado es 0
filtro = (df_normalizado['playtime_forever'] ==0).sum()
filtro

1847730

In [13]:
print(df_normalizado.isnull().sum())

item_id             0
item_name           0
playtime_forever    0
user_id             0
items_count         0
dtype: int64


Se exporta el dataset en formato parquet

In [14]:
df_normalizado.to_parquet('Datasets/user_items.parquet')